In [1]:
import urllib
import zipfile
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential

In [2]:
def preprocess(image, label):
    # NORMALIZE YOUR IMAGES HERE (HINT: Rescale by 1/.255)
    image = tf.cast(image/255. ,tf.float32)
    return image, label

In [3]:
def solution_model():
    # Downloads and extracts the dataset to the directory that
    # contains this file.

    IMG_SIZE = 128
    BATCH_SIZE = 8

    # The following code reads the training and validation data from their
    # respective directories, resizes them into the specified image size
    # and splits them into batches. You must fill in the image_size
    # argument for both training and validation data.
    # HINT: Image size is a tuple
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory='train/',
        image_size=(128, 128),# YOUR CODE HERE
        label_mode='binary', 
        batch_size=BATCH_SIZE)

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory='validation/',
        label_mode='binary', 
        image_size= (128,128),# YOUR CODE HERE
        batch_size=BATCH_SIZE)
    
    normalization_layer = tf.keras.layers.Rescaling(1./255)
    train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

    # Normalizes train and validation datasets using the
    # preprocess() function.
    # Also makes other calls, as evident from the code, to prepare them for
    # training.
    # Do not batch or resize the images in the dataset here since it's already
    # been done previously.
    # train_ds = train_ds.map(
    #     preprocess)
    # val_ds = val_ds.map(
    #     preprocess)

    # Code to define the model
    base = tf.keras.applications.efficientnet.EfficientNetB0(input_shape=(128,128,3),
                                                             weights='imagenet',
                                                             include_top=False)
    base.trainable = False
    model = tf.keras.Sequential([
        base,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])



    # Code to compile and train the model
    model.compile(optimizer=tf.optimizers.Adam(), 
                  loss='binary_crossentropy',
                    metrics='accuracy')

    history = model.fit(
        train_ds,
        batch_size=BATCH_SIZE,
        validation_data=val_ds,
        epochs=10
    )

    return model

In [4]:
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")

Found 10000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
Epoch 1/10
1250/1250 [==============================] - 24s 16ms/step - loss: 0.3295 - accuracy: 0.8905 - val_loss: 0.3445 - val_accuracy: 0.9130
Epoch 2/10
1250/1250 [==============================] - 18s 15ms/step - loss: 0.1751 - accuracy: 0.9476 - val_loss: 0.4426 - val_accuracy: 0.9025
Epoch 3/10
1250/1250 [==============================] - 18s 14ms/step - loss: 0.1268 - accuracy: 0.9577 - val_loss: 0.3859 - val_accuracy: 0.9185
Epoch 4/10
1250/1250 [==============================] - 18s 14ms/step - loss: 0.0987 - accuracy: 0.9687 - val_loss: 0.4247 - val_accuracy: 0.9200
Epoch 5/10
1250/1250 [==============================] - 18s 15ms/step - loss: 0.0949 - accuracy: 0.9699 - val_loss: 0.4303 - val_accuracy: 0.9265
Epoch 6/10
1250/1250 [==============================] - 18s 14ms/step - loss: 0.0920 - accuracy: 0.9718 - val_loss: 0.4869 - val_accuracy: 0.9190
Epoch 7/10
1250/1250 [===================

In [ ]:
print(history.history)